# The Reality Filter

> Without Pretrained Model

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Load images

In [3]:
class ImageDataset(Dataset):
    def __init__(self, dir_, csv_filename=None, aggressive=False):
        super().__init__()
        if aggressive:
            self.transform = transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
                transforms.ToTensor()
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])

        self.dir_ = Path(dir_)
        assert self.dir_.exists() and self.dir_.is_dir(), "image directory doesn't exist"

        self.images = sorted(self.dir_.glob("*.jpg"))
        self.labels = None
        if csv_filename is not None:
            self.labels = pd.read_csv(csv_filename)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_filename = self.images[idx]
        img = self.transform(Image.open(img_filename).convert("RGB"))

        label = torch.nan
        if self.labels is not None:
            label = self.labels.loc[self.labels["CodeID"] == int(img_filename.stem), "Label"].values[0]
        
        return img, label

In [4]:
ds_train = ImageDataset("./train", "./train_data.csv", aggressive=True)
ds_test = ImageDataset("./test")

In [5]:
dl_train = DataLoader(ds_train, batch_size=32)
dl_test = DataLoader(ds_test, batch_size=32)

## Subtask 1

In [6]:
img129 = Image.open("./train/129.jpg").convert("RGB")
width, height = img129.size
width, height

(728, 486)

In [7]:
subtask1_rows = [(1, 0, width * height)]

## Subtask 2

In [8]:
labels = ds_train.labels["Label"]
zeros, ones = labels[labels == 0].count().item(), labels[labels == 1].count().item()
zeros, ones

(130, 93)

In [9]:
len(labels)

223

In [10]:
subtask2_rows = [(2, 0, min(zeros, ones) / max(zeros, ones))]

## Define model

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7)),  # Makes output shape fixed
            nn.Dropout(0.25)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.fc(x)
        return x

In [12]:
def train(model, dataloader, criterion, optimizer, device):
    model = model.to(device)
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.float().to(device), labels.float().to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze(-1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = outputs.round().to(torch.uint8)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(dataloader), correct / total

In [13]:
@torch.no_grad()
def predict(model, dataloader, device):
    model = model.to(device)
    model.eval()

    all_preds = []
    for images, labels in dataloader:
        images = images.float().to(device)
        outputs = model(images).squeeze(-1)
        preds = outputs.round().to(torch.uint8)
        all_preds.extend(preds.cpu().tolist())
    
    return all_preds

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

In [15]:
for epoch in range(10):
    loss, accuracy = train(model, dl_train, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}: {loss=}, {accuracy=}")

Epoch 1: loss=1.7662346618516105, accuracy=0.4080717488789238
Epoch 2: loss=0.6254737888063703, accuracy=0.4977578475336323
Epoch 3: loss=0.6849923985345023, accuracy=0.6233183856502242
Epoch 4: loss=0.6829503178596497, accuracy=0.5739910313901345
Epoch 5: loss=0.6779274259294782, accuracy=0.5919282511210763
Epoch 6: loss=0.6880688496998378, accuracy=0.5829596412556054
Epoch 7: loss=0.6816920297486442, accuracy=0.5829596412556054
Epoch 8: loss=0.669362621647971, accuracy=0.5739910313901345
Epoch 9: loss=0.681648748261588, accuracy=0.5829596412556054
Epoch 10: loss=0.6873103124754769, accuracy=0.57847533632287


## Subtask 3

In [16]:
predictions = predict(model, dl_test, device)
ids = [int(img.stem) for img in ds_test.images]

subtask3_rows = []
for id_, pred in zip(ids, predictions):
    subtask3_rows.append((3, id_, pred))

C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


## Save answers

In [17]:
submission_rows = subtask1_rows + subtask2_rows + subtask3_rows
df_submission = pd.DataFrame(submission_rows, columns=["subtaskID", "datapointID", "answer"])
df_submission.to_csv("submission.csv", index=False)

## Submission results

Subtask 1:
- Accuracy: 1
- Score: 10/10

Subtask 2:
- Accuracy: 1
- Score: 10/10

Subtask 3:
- Accuracy: 0.529914
- Score: 40/80